<h2 align="center">
<font color='green'>    Vale3 - Stock Analisys - Data Preparation</font></h2>   <img src=logo-vale-escudo-256.png style="float: left; width:80px; height:80px;" />


<h4 align="center"><font color='gray'>Data Preparation - Capture vale3 informationa from different sources, different periodicities to create a consistent data repositorie that will support the different algothms</font></h4>


<h4> <align="center"><font color='orange'> Index : </font></h4>
    

- [0 - Librarys](#0)
- [1 - Functions](#1)
- [2 - Daily Stock Prices](#2)
- [3 - Dividends](#3) 
- [4 - JSCP](#5) 
- [5 - SP500](#6) 
- [6 - Income Information](#7) 
- [7 - Balance Information](#8) 
- [12 - Graficos e Analises](#9) 


<h4> <align="center"><font color='yellow'> Ajustes no código : </font></h4>

- Passamos a deixar a data como índice em todos os arquivos 
- passamos a extrair as informações de dividendos diretamente no YFinance, sem upload de arquivo

 ### 0 - Librarys

In [323]:
# Importando as bibliotecas necessárias

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# Configuranco o periodo de extração das informações
# 

start = '2008-01-01'
end = '2023-10-31'


### 1 - Functions

In [324]:
# funcao para extrair informacoes do income
def extract_financial_info(year, column_name, dataframe):
    """
    Função para extrair informações financeiras de um DataFrame com base no ano e no nome da coluna.
    
    Parâmetros:
    - year (str): Ano para o qual a informação deve ser extraída.
    - column_name (str): Nome da coluna (informação financeira) que se deseja extrair.
    - dataframe (pd.DataFrame): DataFrame contendo os dados financeiros.
    
    Retorna:
    - Valor da informação financeira para o ano especificado.
    """
    try:
        value = income[year][column_name]
        return value
    except KeyError:
        return f'Informação {column_name} para o ano {year} não encontrada.'
    except Exception as e:
        return f'Erro ao extrair informação: {e}'


In [325]:
def inserir_income(df, EBITDA, valor, atualizar=False):
  """
  Insere informações do balanço, que tem periodicidade anual, no arquivo da vale que .

  Args:
    df: O DataFrame no qual a coluna será inserida.
    nome_coluna: O nome da coluna a ser inserida.
    valor: O valor a ser inserido na coluna.
    atualizar: Se True, o valor será atualizado na coluna se ela já existir. Se False, a coluna será criada se não existir.

  Returns:
    O DataFrame com a coluna inserida.
  """

  # Verifica se a coluna já existe

  if nome_coluna in df.columns:
    # Atualiza o valor da coluna
    if atualizar:
      df[nome_coluna] = valor
    else:
      raise ValueError(f"A coluna '{nome_coluna}' já existe. Use atualizar=True para atualizar o valor da coluna.")
  else:
    # Cria a coluna
    df[nome_coluna] = valor

  return df

In [326]:
def nulls_to_zero(df):
    """
    Substitui valores nulos em um DataFrame do pandas por zero.
    
    Args:
    - df (pd.DataFrame): DataFrame de entrada.


    Returns:
    - pd.DataFrame: DataFrame com valores nulos substituídos por zero.
    - str: Mensagem informando a quantidade de valores convertidos ou um erro.
    """
    try:
        # Contando a quantidade de valores nulos antes da substituição
        null_count = df.isnull().sum().sum()
        
        # Substituindo valores nulos por zero
        df.fillna(0, inplace=True)
        
        message = f"{null_count} valores nulos foram substituídos por zero."
        return df, message
    except Exception as e:
        return df, f"Ocorreu um erro: {str(e)}"



In [327]:
# Verificando linhas duplicadas em um determinado arquivo txt, csv, etc
def file_linhas_duplicadas(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
    line_counts = {}  # Dicionário para armazenar a contagem de cada linha
    for line in lines:
        line = line.strip()  # Removendo espaços em branco no início e no fim
        line_counts[line] = line_counts.get(line, 0) + 1  # Incrementando a contagem
    
    # Calculando o número de linhas duplicadas
    duplicate_lines_count = sum(count - 1 for count in line_counts.values() if count > 1)
    
    return duplicate_lines_count

In [328]:
"""
    Esta função verifica se um DataFrame tem linhas duplicadas.
    
    Parâmetros:
    df (pandas.DataFrame): O DataFrame a ser verificado.
    
    Retorno:
    int: O número de linhas duplicadas no DataFrame.
"""

def df_linhas_duplicadas(df):
   
    linhas_duplicadas = df.duplicated().sum()
    return linhas_duplicadas


In [329]:

def verifica_nulos(df):
    """
    Esta função verifica se um DataFrame tem informações nulas.
    
    Parâmetros:
    df (pandas.DataFrame): O DataFrame a ser verificado.
    
    Retorno:
    pandas.DataFrame ou str: As linhas com informações nulas ou uma mensagem indicando que não há nulos.
    """
    # Verifica se há alguma célula nula no DataFrame
    if df.isnull().values.any():
        # Retorna as linhas que têm algum valor nulo
        linhas_nulas = df[df.isnull().any(axis=1)]
        return linhas_nulas
    else:
        # Retorna uma mensagem indicando que não há valores nulos
        return "Não existem valores nulos no DataFrame."

# Exemplo de uso:
# Suponha que df seja o DataFrame que você quer verificar.
# resultado = verifica_nulos(df)
# print(resultado)


In [330]:
# Funcao para gerar um arquivo csv para alimentar os diversos modelos que serao utilizados
# no projeto

def gera_csv(df, nome_arquivo='dados.csv'):
    """
    Esta função recebe um DataFrame pandas como parâmetro e gera um arquivo CSV.
    
    Parâmetros:
    df (pandas.DataFrame): O DataFrame a ser convertido em CSV.
    nome_arquivo (str): O nome do arquivo CSV a ser gerado. O padrão é 'dados.csv'.
    
    Retorno:
    str: Uma mensagem indicando sucesso ou falha na geração do arquivo CSV.
    """
    try:
        # Gera o arquivo CSV
        df.to_csv(nome_arquivo, index=False)
        return f'Arquivo {nome_arquivo} gerado com sucesso.'
    except Exception as e:
        # Retorna uma mensagem de erro se algo der errado
        return f'Ocorreu um erro ao gerar o arquivo CSV: {str(e)}'

# Exemplo de uso:
# Suponha que df seja o DataFrame que você quer converter em CSV.
# mensagem = gera_csv(df, 'meu_arquivo.csv')
# print(mensagem)



In [331]:
def inserir_income(df, EBITDA, valor, atualizar=False):
  """
  Insere informações do balanço, que tem periodicidade anual, no arquivo da vale que .

  Args:
    df: O DataFrame no qual a coluna será inserida.
    nome_coluna: O nome da coluna a ser inserida.
    valor: O valor a ser inserido na coluna.
    atualizar: Se True, o valor será atualizado na coluna se ela já existir. Se False, a coluna será criada se não existir.

  Returns:
    O DataFrame com a coluna inserida.
  """

  # Verifica se a coluna já existe

  if nome_coluna in df.columns:
    # Atualiza o valor da coluna
    if atualizar:
      df[nome_coluna] = valor
    else:
      raise ValueError(f"A coluna '{nome_coluna}' já existe. Use atualizar=True para atualizar o valor da coluna.")
  else:
    # Cria a coluna
    df[nome_coluna] = valor

  return df

 ### 2 - Daily Stock Prices

<hr>
<p> data = 07/11/2023 </p> 
<p> - Foram extraidas informaçoes de 3 fontes de dados, foi utilizado o comando parse_date e set_index, para criar as datas como indice e fazer o merge dos arquivos
<p> - Foram trabalhadas as informacoes de nulos
 

 </p>   
<hr>

In [332]:
# Definindo a ativo que vamos extrair informações
#
symbol = 'vale3.sa'

In [388]:
# get historical market data
vale = yf.download(symbol, start, end)

[*********************100%%**********************]  1 of 1 completed


In [334]:
vale.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3930 entries, 2008-01-02 to 2023-10-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3930 non-null   float64
 1   High       3930 non-null   float64
 2   Low        3930 non-null   float64
 3   Close      3930 non-null   float64
 4   Adj Close  3930 non-null   float64
 5   Volume     3930 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 214.9 KB


In [389]:
# a data ficou formatada com HH:MM:SS, ficou ruim de trabalhar nas queries
vale.head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-02,59.250000,59.290001,57.549999,57.900002,26.739393,1200600
2008-01-03,58.060001,58.860001,57.889999,58.200001,26.877943,1114700


In [390]:
vale['Datebkp'] = vale.index
vale.head(2)

,Open,High,Low,Close,Adj Close,Volume,Datebkp
Date,,,,,,,
2008-01-02,59.250000,59.290001,57.549999,57.900002,26.739393,1200600,2008-01-02
2008-01-03,58.060001,58.860001,57.889999,58.200001,26.877943,1114700,2008-01-03


In [391]:
vale.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3930 entries, 2008-01-02 to 2023-10-30
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Open       3930 non-null   float64       
 1   High       3930 non-null   float64       
 2   Low        3930 non-null   float64       
 3   Close      3930 non-null   float64       
 4   Adj Close  3930 non-null   float64       
 5   Volume     3930 non-null   int64         
 6   Datebkp    3930 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 245.6 KB


In [338]:
# Ajustando a data 
# Converter o campo de data em uma situação em que a data não é indice
#vale["Datebkp"] = vale["Datebkp"].apply(lambda x: x.strftime("%Y-%m-%d"))
#vale.head()

In [392]:
############################################################################################
# Deleting some columns from the dataframe
############################################################################################
del vale['Open']
del vale['High']    
del vale['Low']
del vale['Adj Close'] 
del vale['Volume'] 
#del vale['Dividends'] 
vale.head()

,Close,Datebkp
Date,,
2008-01-02,57.900002,2008-01-02
2008-01-03,58.200001,2008-01-03
2008-01-04,56.099998,2008-01-04
2008-01-07,55.099998,2008-01-07
2008-01-08,56.200001,2008-01-08


In [ ]:
######################################################################################################
# Grafico em cima do preco de fechamento 'Close'                                                                #
# dataCom(NEGOCIADO ATÉ) - ate esta data quem estiver com o papel em carteira vai receber dividendos  #
#                                                                                                     #
#######################################################################################################

plt.figure(figsize=(14, 7))
plt.plot(vale.index, vale['Close'], label='Close', color='blue')
plt.title('Gráfico de Linha - Close x Date')
plt.xlabel('Date')
plt.ylabel('Close')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<p> <align="center"><font color='white'>Creating a new columns 'tomorrow' and 'target' to help in the prediction process</font></p>

In [393]:
# queremos mostrar se existe uma tendencia de alta, para isto vamos criar uma coluna 'tomorrow', vamos prever o amanha
# vamos utilizar o metodo shift do Pandas

vale['Tomorrow'] = vale['Close'].shift(-1)

In [394]:
vale.head(2)

,Close,Datebkp,Tomorrow
Date,,,
2008-01-02,57.900002,2008-01-02,58.200001
2008-01-03,58.200001,2008-01-03,56.099998


In [395]:
# queremos prever que tomorrow's price seja maior que today's price
# tambem vamos converter os campos para inteiro usando a funcao astype(int)
vale['Target'] = (vale['Tomorrow'] > vale['Close']).astype(int)

In [396]:
vale.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3930 entries, 2008-01-02 to 2023-10-30
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Close     3930 non-null   float64       
 1   Datebkp   3930 non-null   datetime64[ns]
 2   Tomorrow  3929 non-null   float64       
 3   Target    3930 non-null   int32         
dtypes: datetime64[ns](1), float64(2), int32(1)
memory usage: 138.2 KB


In [397]:
vale.index

DatetimeIndex(['2008-01-02', '2008-01-03', '2008-01-04', '2008-01-07',
               '2008-01-08', '2008-01-09', '2008-01-10', '2008-01-11',
               '2008-01-14', '2008-01-15',
               ...
               '2023-10-17', '2023-10-18', '2023-10-19', '2023-10-20',
               '2023-10-23', '2023-10-24', '2023-10-25', '2023-10-26',
               '2023-10-27', '2023-10-30'],
              dtype='datetime64[ns]', name='Date', length=3930, freq=None)

### 3 -  Dividends: Loading Data from YFinance
<hr>

In [378]:
# vamos extrair as informaçoes utilizando o metodo ticker

valeDiv = yf.Ticker("vale3.sa")

In [384]:
# Extraindo as informações da API utilizando o metodo Ticker

dividendos = valeDiv.dividends
dividendos

Date
2008-04-11 00:00:00-03:00    0.436585
2008-10-17 00:00:00-03:00    0.138520
2009-04-16 00:00:00-03:00    0.524586
2009-10-16 00:00:00-03:00    0.492036
2010-04-15 00:00:00-03:00    0.421661
2010-10-15 00:00:00-03:00    0.471881
2011-01-17 00:00:00-02:00    0.320048
2011-04-14 00:00:00-03:00    0.608247
2011-08-12 00:00:00-03:00    0.933403
2011-10-17 00:00:00-02:00    0.634329
2012-04-16 00:00:00-03:00    1.075280
2012-10-17 00:00:00-03:00    1.186520
2013-04-17 00:00:00-03:00    0.864045
2013-10-18 00:00:00-03:00    0.120605
2014-04-15 00:00:00-03:00    0.898904
2014-10-17 00:00:00-03:00    0.990870
2015-04-15 00:00:00-03:00    0.601761
2015-10-16 00:00:00-03:00    0.373610
2016-12-02 00:00:00-02:00    0.166294
2017-04-24 00:00:00-03:00    0.905572
2017-12-22 00:00:00-02:00    0.419912
2018-03-07 00:00:00-03:00    0.488511
2018-08-03 00:00:00-03:00    1.480362
2019-12-27 00:00:00-03:00    1.414364
2020-09-22 00:00:00-03:00    2.407510
2021-03-05 00:00:00-03:00    4.261646
2021-06

In [380]:
# O Ticker extrai uma série pandas com as informações de dividendos
dividendos.info()


<class 'pandas.core.series.Series'>
DatetimeIndex: 33 entries, 2008-04-11 00:00:00-03:00 to 2023-08-14 00:00:00-03:00
Series name: Dividends
Non-Null Count  Dtype  
--------------  -----  
33 non-null     float64
dtypes: float64(1)
memory usage: 528.0 bytes


In [385]:
dividendos.index

DatetimeIndex(['2008-04-11 00:00:00-03:00', '2008-10-17 00:00:00-03:00',
               '2009-04-16 00:00:00-03:00', '2009-10-16 00:00:00-03:00',
               '2010-04-15 00:00:00-03:00', '2010-10-15 00:00:00-03:00',
               '2011-01-17 00:00:00-02:00', '2011-04-14 00:00:00-03:00',
               '2011-08-12 00:00:00-03:00', '2011-10-17 00:00:00-02:00',
               '2012-04-16 00:00:00-03:00', '2012-10-17 00:00:00-03:00',
               '2013-04-17 00:00:00-03:00', '2013-10-18 00:00:00-03:00',
               '2014-04-15 00:00:00-03:00', '2014-10-17 00:00:00-03:00',
               '2015-04-15 00:00:00-03:00', '2015-10-16 00:00:00-03:00',
               '2016-12-02 00:00:00-02:00', '2017-04-24 00:00:00-03:00',
               '2017-12-22 00:00:00-02:00', '2018-03-07 00:00:00-03:00',
               '2018-08-03 00:00:00-03:00', '2019-12-27 00:00:00-03:00',
               '2020-09-22 00:00:00-03:00', '2021-03-05 00:00:00-03:00',
               '2021-06-24 00:00:00-03:00', '2021-0

In [382]:
#Ajustando o formato da data do index que está com hh:mm:ss para ficar compativel com o indice do data frame

dividendos.index = dividendos.index.strftime("%Y-%m-%d")

dividendos.head()

Date
2008-04-11    0.436585
2008-10-17    0.138520
2009-04-16    0.524586
2009-10-16    0.492036
2010-04-15    0.421661
Name: Dividends, dtype: float64

In [383]:
dividendos.index


Index(['2008-04-11', '2008-10-17', '2009-04-16', '2009-10-16', '2010-04-15',
       '2010-10-15', '2011-01-17', '2011-04-14', '2011-08-12', '2011-10-17',
       '2012-04-16', '2012-10-17', '2013-04-17', '2013-10-18', '2014-04-15',
       '2014-10-17', '2015-04-15', '2015-10-16', '2016-12-02', '2017-04-24',
       '2017-12-22', '2018-03-07', '2018-08-03', '2019-12-27', '2020-09-22',
       '2021-03-05', '2021-06-24', '2021-09-23', '2022-03-09', '2022-08-12',
       '2022-12-13', '2023-03-14', '2023-08-14'],
      dtype='object', name='Date')

In [386]:
dividendos.index = pd.to_datetime(dividendos.index).date
dividendos.index


Index([2008-04-11, 2008-10-17, 2009-04-16, 2009-10-16, 2010-04-15, 2010-10-15,
       2011-01-17, 2011-04-14, 2011-08-12, 2011-10-17, 2012-04-16, 2012-10-17,
       2013-04-17, 2013-10-18, 2014-04-15, 2014-10-17, 2015-04-15, 2015-10-16,
       2016-12-02, 2017-04-24, 2017-12-22, 2018-03-07, 2018-08-03, 2019-12-27,
       2020-09-22, 2021-03-05, 2021-06-24, 2021-09-23, 2022-03-09, 2022-08-12,
       2022-12-13, 2023-03-14, 2023-08-14],
      dtype='object')

In [398]:
######################################################################################
#  Utilizando o método join para criar uma nova coluna no data frame com informaçoes de dividendos
######################################################################################

vale = vale.join(dividendos.rename('dividends'))
vale

,Close,Datebkp,Tomorrow,Target,dividends
Date,,,,,
2008-01-02,57.900002,2008-01-02,58.200001,1,NaN
2008-01-03,58.200001,2008-01-03,56.099998,0,NaN
2008-01-04,56.099998,2008-01-04,55.099998,0,NaN
2008-01-07,55.099998,2008-01-07,56.200001,1,NaN
2008-01-08,56.200001,2008-01-08,55.599998,0,NaN
...,...,...,...,...,...
2023-10-24,63.990002,2023-10-24,63.930000,0,NaN
2023-10-25,63.930000,2023-10-25,65.300003,1,NaN
2023-10-26,65.300003,2023-10-26,67.570000,1,NaN


In [ ]:
# Olha que interessante, o join alterou o padrao de datatime do indice

vale.index

In [399]:
# Verificando se as informaçoes de dividendos foram carregadas corretamente
vale[vale['Datebkp'] == '2009-04-16']

,Close,Datebkp,Tomorrow,Target,dividends
Date,,,,,
2009-04-16,35.59,2009-04-16,35.299999,0,0.524586


In [ ]:

# Merge the 'Aprovado' information into your DataFrame
#vale_div = vale.merge(dividendos[['Dividends']], left_index=True, right_on='Dividends', how='left')
#df_merged = df1.merge(df2, left_index=True, right_index=True, how='inner')


#vale_d = vale.merge(dividendos, left_index=True, right_index=True, how='left')

#vale_d.head(2)

In [401]:
vale[vale.index == '2019-12-27']

,Close,Datebkp,Tomorrow,Target,dividends
Date,,,,,
2019-12-27,53.599998,2019-12-27,53.299999,0,1.414364


### 4 - JCSP


 <p style="font-size: 16px;">Loading, cleaning and quality check for JCSP information !</p>
 <p style="font-size: 12px;">Source: YFinance</p>
 <p style="font-size: 12px;">Type: File extracted Manually</p>
 <p style="font-size: 12px;">Obs. Choose file against API because the format were more suitable for the stretegy</p>

In [405]:
#######################################################################################################
# Carregando informacoes de JSCP	                                                              #
# dataCom(NEGOCIADO ATÉ) - ate esta data quem estiver com o papel em carteira vai receber dividendos  #
# APROVADO - data de aprovacao do pagamento do dividendo                                              #
# VALOR (R$) - Valor por acao que sera pago apurado conforme balanco                                  #
#######################################################################################################

JSCP = pd.read_csv('Vale3-ProventosGeral.csv', parse_dates=['APROVADO'], index_col='APROVADO')
JSCP

C:\Users\edmun\AppData\Local\Temp\ipykernel_21132\2487070507.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  JSCP = pd.read_csv('Vale3-ProventosGeral.csv', parse_dates=['APROVADO'], index_col='APROVADO')


,TIPO,VALOR (R$),% / FATOR,VALOR EMISSÃO (R$),NEGOCIADO ATÉ,PAGAMENTO
APROVADO,,,,,,
2023-07-27,JSCP,"1,91",n/d,n/d,11/08/23,01/09/23
2023-02-16,Dividendo,"1,82",n/d,n/d,13/03/23,22/03/23
2022-01-12,JSCP,"0,290809381",n/d,n/d,12/12/22,n/d
2022-07-28,Dividendo,"2,03",n/d,n/d,11/08/22,01/09/22
2022-07-28,JSCP,"1,53",n/d,n/d,11/08/22,01/09/22
2022-02-24,Dividendo,"3,71",n/d,n/d,08/03/22,16/03/22
2021-09-16,Dividendo,"8,19",n/d,n/d,22/09/21,30/09/21
2021-06-17,Dividendo,"0,716268047",n/d,n/d,23/06/21,30/06/21
2021-06-17,Dividendo,"1,46",n/d,n/d,23/06/21,30/06/21


In [ ]:
JSCP.info()


In [406]:
# criando um data frame com as informacoes de JSCP apenas
JSCP = JSCP[JSCP['TIPO'] == 'JSCP']
JSCP


,TIPO,VALOR (R$),% / FATOR,VALOR EMISSÃO (R$),NEGOCIADO ATÉ,PAGAMENTO
APROVADO,,,,,,
2023-07-27,JSCP,"1,91",n/d,n/d,11/08/23,01/09/23
2022-01-12,JSCP,"0,290809381",n/d,n/d,12/12/22,n/d
2022-07-28,JSCP,"1,53",n/d,n/d,11/08/22,01/09/22
2021-02-25,JSCP,"0,835736883",n/d,n/d,04/03/21,15/03/21
2020-10-09,JSCP,"0,997344547",n/d,n/d,21/09/20,30/09/20
2019-12-19,JSCP,"1,41",n/d,n/d,26/12/19,n/d


<p> <font color='red'> Tirando as linhas com n/d, basicamente são da data de pagamento 2 linhas</p>

In [408]:
# Deletando as colunas que nao vamos utilizar
del JSCP['TIPO']
del JSCP['% / FATOR']
del JSCP['VALOR EMISSÃO (R$)']

JSCP

,VALOR (R$),NEGOCIADO ATÉ,PAGAMENTO
APROVADO,,,
2023-07-27,"1,91",11/08/23,01/09/23
2022-01-12,"0,290809381",12/12/22,n/d
2022-07-28,"1,53",11/08/22,01/09/22
2021-02-25,"0,835736883",04/03/21,15/03/21
2020-10-09,"0,997344547",21/09/20,30/09/20
2019-12-19,"1,41",26/12/19,n/d


In [409]:
JSCP = JSCP.drop(pd.Timestamp('2022-01-12'))

In [410]:
JSCP = JSCP.drop(pd.Timestamp('2019-12-19'))

In [411]:
JSCP[JSCP['PAGAMENTO'] == 'n/d']

,VALOR (R$),NEGOCIADO ATÉ,PAGAMENTO
APROVADO,,,


In [412]:
############################################################################################
# Convertendo a coluna 'data_obj' para datetime no formato do indice do arquivo dividendos #
# para deixar no mesmo formato dos campos do arquivo de vale                               #
############################################################################################

JSCP['PAGAMENTO'] = pd.to_datetime(JSCP['PAGAMENTO'], format='%d/%m/%y')
JSCP['NEGOCIADO ATÉ'] = pd.to_datetime(JSCP['NEGOCIADO ATÉ'], format='%d/%m/%y')
JSCP

,VALOR (R$),NEGOCIADO ATÉ,PAGAMENTO
APROVADO,,,
2023-07-27,"1,91",2023-08-11,2023-09-01
2022-07-28,"1,53",2022-08-11,2022-09-01
2021-02-25,"0,835736883",2021-03-04,2021-03-15
2020-10-09,"0,997344547",2020-09-21,2020-09-30


In [413]:
# Renomenado a coluna para jscpDataCom
JSCP.rename(columns={
    'VALOR (R$)': 'jscpValor',
    'NEGOCIADO ATÉ': 'jscpDataCom',
    'PAGAMENTO': 'jscpPagamento'
    }, inplace=True)
JSCP

,jscpValor,jscpDataCom,jscpPagamento
APROVADO,,,
2023-07-27,"1,91",2023-08-11,2023-09-01
2022-07-28,"1,53",2022-08-11,2022-09-01
2021-02-25,"0,835736883",2021-03-04,2021-03-15
2020-10-09,"0,997344547",2020-09-21,2020-09-30


In [402]:
JSCP.head(2)

,jscpValor,jscpDataCom,jscpPagamento
APROVADO,,,
2023-07-27,"1,91",11/08/23,2023-09-01
2022-07-28,"1,53",11/08/22,2022-09-01


In [414]:
# Merge the 'Aprovado' information into your DataFrame

vale_dj = vale.merge(JSCP, left_index=True, right_index=True, how='left')
vale_dj.head()

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento
Date,,,,,,,,
2008-01-02,57.900002,2008-01-02,58.200001,1,NaN,NaN,NaT,NaT
2008-01-03,58.200001,2008-01-03,56.099998,0,NaN,NaN,NaT,NaT
2008-01-04,56.099998,2008-01-04,55.099998,0,NaN,NaN,NaT,NaT
2008-01-07,55.099998,2008-01-07,56.200001,1,NaN,NaN,NaT,NaT
2008-01-08,56.200001,2008-01-08,55.599998,0,NaN,NaN,NaT,NaT


In [415]:
# Verificando se o merge foi feito com sucesso

vale_dj[vale_dj.index == '2023-07-27']

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento
Date,,,,,,,,
2023-07-27,70.419998,2023-07-27,67.629997,0,NaN,"1,91",2023-08-11,2023-09-01


In [417]:
#####################################################################
# Convertendo o NaT para 0 e o valor de data para 1                 # 
#####################################################################
# sugestao do chatgpt para resolver o problema
#
vale_dj['dividends'] = vale_dj['dividends'].notna().astype(int)
vale_dj['jscpValor'] = vale_dj['jscpValor'].notna().astype(int)
vale_dj['jscpDataCom'] = vale_dj['jscpDataCom'].notna().astype(int)
vale_dj['jscpPagamento'] = vale_dj['jscpPagamento'].notna().astype(int)
vale_dj.head()

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento
Date,,,,,,,,
2008-01-02,57.900002,2008-01-02,58.200001,1,0,0,0,0
2008-01-03,58.200001,2008-01-03,56.099998,0,0,0,0,0
2008-01-04,56.099998,2008-01-04,55.099998,0,0,0,0,0
2008-01-07,55.099998,2008-01-07,56.200001,1,0,0,0,0
2008-01-08,56.200001,2008-01-08,55.599998,0,0,0,0,0


In [418]:
verifica_nulos(vale_dj)

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento
Date,,,,,,,,
2023-10-30,68.18,2023-10-30,NaN,0,0,0,0,0


In [419]:
# Deletando a linha com tomorrow = NaN
vale_dj = vale_dj.drop(pd.Timestamp('2023-10-30'))

<h4> <font color='red'> Deletando a Informaçao de Volume que está trazendo um dado enviazado que pode atrapalhar o algoritmo</h4>

### 5 - S&P500

In [420]:
# Definindo o ativo
symbol = '^GSPC'

In [421]:
# get historical market data
sp500 = yf.download(symbol, start, end)

[*********************100%%**********************]  1 of 1 completed


In [422]:
sp500.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-02,1467.969971,1471.770020,1442.069946,1447.160034,1447.160034,3452650000
2008-01-03,1447.550049,1456.800049,1443.729980,1447.160034,1447.160034,3429500000
2008-01-04,1444.010010,1444.010010,1411.189941,1411.630005,1411.630005,4166000000


In [423]:
sp500.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3985 entries, 2008-01-02 to 2023-10-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3985 non-null   float64
 1   High       3985 non-null   float64
 2   Low        3985 non-null   float64
 3   Close      3985 non-null   float64
 4   Adj Close  3985 non-null   float64
 5   Volume     3985 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 217.9 KB


In [424]:
############################################################################################
# Deleting some columns from the dataframe
############################################################################################
del sp500['Open']
del sp500['High']    
del sp500['Low']
del sp500['Adj Close'] 
del sp500['Volume'] 

sp500.head(2)

,Close
Date,
2008-01-02,1447.160034
2008-01-03,1447.160034


In [425]:
# Renomenado a coluna para sp500
sp500.rename(columns={
    'Close': 'sp500'
    }, inplace=True)
sp500.head(2)

,sp500
Date,
2008-01-02,1447.160034
2008-01-03,1447.160034


In [426]:
# Merge the 'Aprovado' information into your DataFrame

vale_dj = vale_dj.merge(sp500, left_index=True, right_index=True, how='left')
vale_dj.head()

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento,sp500
Date,,,,,,,,,
2008-01-02,57.900002,2008-01-02,58.200001,1,0,0,0,0,1447.160034
2008-01-03,58.200001,2008-01-03,56.099998,0,0,0,0,0,1447.160034
2008-01-04,56.099998,2008-01-04,55.099998,0,0,0,0,0,1411.630005
2008-01-07,55.099998,2008-01-07,56.200001,1,0,0,0,0,1416.180054
2008-01-08,56.200001,2008-01-08,55.599998,0,0,0,0,0,1390.189941


In [ ]:
# exportando o modelo para rodar os algoritmos
#gera_csv(vale, nome_arquivo='vale_dj.csv', )

### 6 - Dolar Comercial

In [427]:
# Definindo o ativo
symbol = "USDBRL=X"

In [429]:
# get historical market data
dolar = yf.download(symbol, start, end)
dolar.tail(2)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-10-27,4.98530,4.9874,4.93020,4.98530,4.98530,0
2023-10-30,4.95203,5.0478,4.95203,4.95203,4.95203,0


In [430]:
############################################################################################
# Deleting some columns from the dataframe
############################################################################################
del dolar['Open']
del dolar['High']    
del dolar['Low']
del dolar['Adj Close'] 
del dolar['Volume'] 

dolar.head(2)

,Close
Date,
2008-01-01,1.7673
2008-01-02,1.7645


In [431]:
# Renomenado a coluna para sp500
dolar.rename(columns={
    'Close': 'US$Comercial'
    }, inplace=True)
dolar.head(2)

,US$Comercial
Date,
2008-01-01,1.7673
2008-01-02,1.7645


In [ ]:
dolar.info()

In [432]:
# Merge the 'Aprovado' information into your DataFrame

vale_djus = vale_dj.merge(dolar, left_index=True, right_index=True, how='left')
vale_djus.tail()

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento,sp500,US$Comercial
Date,,,,,,,,,,
2008-01-02,57.900002,2008-01-02,58.200001,1,0,0,0,0,1447.160034,1.7645
2008-01-03,58.200001,2008-01-03,56.099998,0,0,0,0,0,1447.160034,1.7480
2008-01-04,56.099998,2008-01-04,55.099998,0,0,0,0,0,1411.630005,1.7572
2008-01-07,55.099998,2008-01-07,56.200001,1,0,0,0,0,1416.180054,1.7640
2008-01-08,56.200001,2008-01-08,55.599998,0,0,0,0,0,1390.189941,1.7520


In [433]:
vale_djus.tail()

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento,sp500,US$Comercial
Date,,,,,,,,,,
2023-10-23,62.560001,2023-10-23,63.990002,1,0,0,0,0,4217.040039,5.0313
2023-10-24,63.990002,2023-10-24,63.930000,0,0,0,0,0,4247.680176,5.0123
2023-10-25,63.930000,2023-10-25,65.300003,1,0,0,0,0,4186.770020,4.9910
2023-10-26,65.300003,2023-10-26,67.570000,1,0,0,0,0,4137.229980,4.9947
2023-10-27,67.570000,2023-10-27,68.180000,1,0,0,0,0,4117.370117,4.9853


In [ ]:
verifica_nulos(vale_djus)

In [ ]:
# retirar os nulos do arquivo


### 7 - Taxa de juros nos USA

<p style="color: red;">Trocar a API, extrair as informações do FED.</p>


In [434]:
symbol = "^TNX"

In [435]:
# get historical market data
usaTxJuros = yf.download(symbol, start, end)
usaTxJuros

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-02,4.033,4.052,3.892,3.901,3.901,0
2008-01-03,3.892,3.958,3.892,3.901,3.901,0
2008-01-04,3.914,3.990,3.809,3.854,3.854,0
2008-01-07,3.893,3.903,3.833,3.839,3.839,0
2008-01-08,3.873,3.897,3.837,3.840,3.840,0
...,...,...,...,...,...,...
2023-10-24,4.870,4.889,4.827,4.840,4.840,0
2023-10-25,4.851,4.959,4.851,4.953,4.953,0
2023-10-26,4.964,4.983,4.842,4.845,4.845,0


In [436]:
############################################################################################
# Deleting some columns from the dataframe
############################################################################################
del usaTxJuros['Open']
del usaTxJuros['High']    
del usaTxJuros['Low']
del usaTxJuros['Adj Close'] 
del usaTxJuros['Volume'] 

usaTxJuros.head(2)

,Close
Date,
2008-01-02,3.901
2008-01-03,3.901


In [437]:
# Renomenado a coluna para sp500
usaTxJuros.rename(columns={
    'Close': 'usaTxJuros'
    }, inplace=True)
usaTxJuros.head(2)

,usaTxJuros
Date,
2008-01-02,3.901
2008-01-03,3.901


In [438]:
usaTxJuros.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3982 entries, 2008-01-02 to 2023-10-30
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   usaTxJuros  3982 non-null   float64
dtypes: float64(1)
memory usage: 62.2 KB


In [439]:
# Merge the 'Aprovado' information into your DataFrame

vale_djustx = vale_djus.merge(usaTxJuros, left_index=True, right_index=True, how='left')
vale_djustx.head()

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento,sp500,US$Comercial,usaTxJuros
Date,,,,,,,,,,,
2008-01-02,57.900002,2008-01-02,58.200001,1,0,0,0,0,1447.160034,1.7645,3.901
2008-01-03,58.200001,2008-01-03,56.099998,0,0,0,0,0,1447.160034,1.7480,3.901
2008-01-04,56.099998,2008-01-04,55.099998,0,0,0,0,0,1411.630005,1.7572,3.854
2008-01-07,55.099998,2008-01-07,56.200001,1,0,0,0,0,1416.180054,1.7640,3.839
2008-01-08,56.200001,2008-01-08,55.599998,0,0,0,0,0,1390.189941,1.7520,3.840


In [ ]:
vale_djustx.info()

In [440]:
verifica_nulos(vale_djustx)

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento,sp500,US$Comercial,usaTxJuros
Date,,,,,,,,,,,
2008-01-21,46.220001,2008-01-21,47.599998,1,0,0,0,0,NaN,1.8262,NaN
2008-02-18,57.590000,2008-02-18,57.910000,1,0,0,0,0,NaN,1.7325,NaN
2008-04-01,61.750000,2008-04-01,61.099998,0,0,0,0,0,1370.180054,NaN,3.545
2008-05-26,67.500000,2008-05-26,66.190002,0,0,0,0,0,NaN,1.6596,NaN
2008-07-01,55.009998,2008-07-01,52.189999,0,0,0,0,0,1284.910034,NaN,3.992
...,...,...,...,...,...,...,...,...,...,...,...
2023-01-16,92.250000,2023-01-16,92.129997,0,0,0,0,0,NaN,5.0807,NaN
2023-05-29,65.830002,2023-05-29,64.279999,0,0,0,0,0,NaN,4.9928,NaN
2023-06-19,69.400002,2023-06-19,67.610001,0,0,0,0,0,NaN,4.8070,NaN


### 6 - Income

In [443]:
# get historical market data
vale3sa = yf.Ticker("vale3.sa")

In [444]:

# get historical market data
hist = vale3sa.history(period="max")

In [445]:
vale3hist = hist
vale3hist.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5996 entries, 2000-01-03 00:00:00-02:00 to 2023-11-14 00:00:00-03:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          5996 non-null   float64
 1   High          5996 non-null   float64
 2   Low           5996 non-null   float64
 3   Close         5996 non-null   float64
 4   Volume        5996 non-null   int64  
 5   Dividends     5996 non-null   float64
 6   Stock Splits  5996 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 374.8 KB


In [446]:
income = vale3sa.income_stmt

In [447]:
income.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, Tax Effect Of Unusual Items to Operating Revenue
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   2022-12-31 00:00:00  56 non-null     object
 1   2021-12-31 00:00:00  55 non-null     object
 2   2020-12-31 00:00:00  57 non-null     object
 3   2019-12-31 00:00:00  57 non-null     object
dtypes: object(4)
memory usage: 2.2+ KB


In [448]:
# Transpondo o DataFrame
df_transposed = income.T

In [449]:
# Convertendo o índice para datetime e removendo a parte da hora
df_transposed.index = pd.to_datetime(df_transposed.index).date

In [450]:
# Convertendo os valores para numérico
df_transposed = df_transposed.apply(pd.to_numeric, errors='coerce')
df_transposed

,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,...,Research And Development,Selling General And Administration,Selling And Marketing Expense,General And Administrative Expense,Other Gand A,Salaries And Wages,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
2022-12-31,3.340500e+08,0.150000,2.161900e+10,2.227000e+09,2.227000e+09,1.672800e+10,3.171000e+09,2.089800e+10,2.384600e+10,2.067500e+10,...,660000000.0,474000000.0,108000000.0,366000000.0,181000000.0,185000000.0,1.981100e+10,2.402800e+10,4.383900e+10,4.383900e+10
2021-12-31,2.842909e+08,0.158999,3.223200e+10,1.788000e+09,1.788000e+09,2.482100e+10,3.034000e+09,1.873700e+10,3.402000e+10,3.098600e+10,...,549000000.0,439000000.0,107000000.0,332000000.0,162000000.0,170000000.0,3.277300e+10,2.172900e+10,5.450200e+10,5.426800e+10
2020-12-31,-8.207321e+08,0.088147,1.995300e+10,-9.311000e+09,-9.311000e+09,4.881000e+09,3.234000e+09,1.585400e+10,1.064200e+10,7.408000e+09,...,443000000.0,505000000.0,105000000.0,400000000.0,176000000.0,224000000.0,2.097900e+10,1.903900e+10,4.001800e+10,3.972100e+10
2019-12-31,-2.675034e+09,0.214414,1.723300e+10,-1.247600e+10,-1.247600e+10,-1.683000e+09,3.726000e+09,2.118700e+10,4.757000e+09,1.031000e+09,...,443000000.0,487000000.0,92000000.0,339000000.0,158000000.0,181000000.0,1.638300e+10,2.118700e+10,3.757000e+10,3.757000e+10


In [451]:
df_transposed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 2022-12-31 to 2019-12-31
Data columns (total 57 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Tax Effect Of Unusual Items                                 4 non-null      float64
 1   Tax Rate For Calcs                                          4 non-null      float64
 2   Normalized EBITDA                                           4 non-null      float64
 3   Total Unusual Items                                         4 non-null      float64
 4   Total Unusual Items Excluding Goodwill                      4 non-null      float64
 5   Net Income From Continuing Operation Net Minority Interest  4 non-null      float64
 6   Reconciled Depreciation                                     4 non-null      float64
 7   Reconciled Cost Of Revenue                                  4 non-null      floa

In [452]:
df_transposed.index = pd.to_datetime(df_transposed.index)
df_transposed.index

DatetimeIndex(['2022-12-31', '2021-12-31', '2020-12-31', '2019-12-31'], dtype='datetime64[ns]', freq=None)

In [453]:
vale_djustx.index

DatetimeIndex(['2008-01-02', '2008-01-03', '2008-01-04', '2008-01-07',
               '2008-01-08', '2008-01-09', '2008-01-10', '2008-01-11',
               '2008-01-14', '2008-01-15',
               ...
               '2023-10-16', '2023-10-17', '2023-10-18', '2023-10-19',
               '2023-10-20', '2023-10-23', '2023-10-24', '2023-10-25',
               '2023-10-26', '2023-10-27'],
              dtype='datetime64[ns]', name='Date', length=3929, freq=None)

<p> Limpando as colunas que não serão utilizadas </p>

In [454]:

# Removendo as colunas que nao serao utilizadas
colunas_para_remover = df_transposed.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 29, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54]].columns

# Removendo as colunas
df_transposed = df_transposed.drop(colunas_para_remover, axis=1)
df_transposed

,EBITDA,Total Expenses,Net Income,Operating Income,Operating Expense,Gross Profit,Total Revenue,Operating Revenue
2022-12-31,2.384600e+10,2.605400e+10,1.878800e+10,1.778500e+10,2.026000e+09,1.981100e+10,4.383900e+10,4.383900e+10
2021-12-31,3.402000e+10,2.366000e+10,2.244500e+10,3.084200e+10,1.931000e+09,3.277300e+10,5.450200e+10,5.426800e+10
2020-12-31,1.064200e+10,2.146800e+10,4.881000e+09,1.855000e+10,2.429000e+09,2.097900e+10,4.001800e+10,3.972100e+10
2019-12-31,4.757000e+09,2.377500e+10,-1.683000e+09,1.379500e+10,2.588000e+09,1.638300e+10,3.757000e+10,3.757000e+10


<p> juntar as informações financeiras anuais do DataFrame income com as informações diárias do DataFrame vale_djustx, usando o ano como chave para a junção</p>

In [455]:
# Criando um campo de ano no arquivo que será utilizado no merge
vale_djustx["Ano"] = vale_djustx.index.year

In [456]:
vale_djustx.head()

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento,sp500,US$Comercial,usaTxJuros,Ano
Date,,,,,,,,,,,,
2008-01-02,57.900002,2008-01-02,58.200001,1,0,0,0,0,1447.160034,1.7645,3.901,2008
2008-01-03,58.200001,2008-01-03,56.099998,0,0,0,0,0,1447.160034,1.7480,3.901,2008
2008-01-04,56.099998,2008-01-04,55.099998,0,0,0,0,0,1411.630005,1.7572,3.854,2008
2008-01-07,55.099998,2008-01-07,56.200001,1,0,0,0,0,1416.180054,1.7640,3.839,2008
2008-01-08,56.200001,2008-01-08,55.599998,0,0,0,0,0,1390.189941,1.7520,3.840,2008


<p> Preparar o DataFrame income para que ele tenha uma coluna de ano e esteja pronto para a junção.</p>

In [457]:
# Resetando o índice no DataFrame income para trabalhar com as datas como uma coluna
df_transposed.reset_index(inplace=True)

In [458]:
# Convertendo a coluna de data para datetime (caso já não seja)
df_transposed['index'] = pd.to_datetime(df_transposed['index'])

In [459]:
# Extraindo o ano e renomeando a coluna
df_transposed['Ano'] = df_transposed['index'].dt.year
df_transposed.drop('index', axis=1, inplace=True)

In [460]:
df_transposed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EBITDA             4 non-null      float64
 1   Total Expenses     4 non-null      float64
 2   Net Income         4 non-null      float64
 3   Operating Income   4 non-null      float64
 4   Operating Expense  4 non-null      float64
 5   Gross Profit       4 non-null      float64
 6   Total Revenue      4 non-null      float64
 7   Operating Revenue  4 non-null      float64
 8   Ano                4 non-null      int32  
dtypes: float64(8), int32(1)
memory usage: 404.0 bytes


In [461]:
vale_djustx.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3929 entries, 2008-01-02 to 2023-10-27
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Close          3929 non-null   float64       
 1   Datebkp        3929 non-null   datetime64[ns]
 2   Tomorrow       3929 non-null   float64       
 3   Target         3929 non-null   int32         
 4   dividends      3929 non-null   int32         
 5   jscpValor      3929 non-null   int32         
 6   jscpDataCom    3929 non-null   int32         
 7   jscpPagamento  3929 non-null   int32         
 8   sp500          3830 non-null   float64       
 9   US$Comercial   3902 non-null   float64       
 10  usaTxJuros     3827 non-null   float64       
 11  Ano            3929 non-null   int32         
dtypes: datetime64[ns](1), float64(5), int32(6)
memory usage: 436.0 KB


In [462]:
# Realizando a junção
df_combinado = pd.merge(vale_djustx, df_transposed, on='Ano', how='left')

In [463]:
df_combinado

,Close,Datebkp,Tomorrow,Target,dividends,jscpValor,jscpDataCom,jscpPagamento,sp500,US$Comercial,usaTxJuros,Ano,EBITDA,Total Expenses,Net Income,Operating Income,Operating Expense,Gross Profit,Total Revenue,Operating Revenue
0,57.900002,2008-01-02,58.200001,1,0,0,0,0,1447.160034,1.7645,3.901,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58.200001,2008-01-03,56.099998,0,0,0,0,0,1447.160034,1.7480,3.901,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56.099998,2008-01-04,55.099998,0,0,0,0,0,1411.630005,1.7572,3.854,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55.099998,2008-01-07,56.200001,1,0,0,0,0,1416.180054,1.7640,3.839,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,56.200001,2008-01-08,55.599998,0,0,0,0,0,1390.189941,1.7520,3.840,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3924,62.560001,2023-10-23,63.990002,1,0,0,0,0,4217.040039,5.0313,4.838,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3925,63.990002,2023-10-24,63.930000,0,0,0,0,0,4247.680176,5.0123,4.840,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3926,63.930000,2023-10-25,65.300003,1,0,0,0,0,4186.770020,4.9910,4.953,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3927,65.300003,2023-10-26,67.570000,1,0,0,0,0,4137.229980,4.9947,4.845,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Verificando se o merge foi feito com sucesso
df_combinado[df_combinado['Ano'] == 2022]

In [465]:
df_combinado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3929 entries, 0 to 3928
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Close              3929 non-null   float64       
 1   Datebkp            3929 non-null   datetime64[ns]
 2   Tomorrow           3929 non-null   float64       
 3   Target             3929 non-null   int32         
 4   dividends          3929 non-null   int32         
 5   jscpValor          3929 non-null   int32         
 6   jscpDataCom        3929 non-null   int32         
 7   jscpPagamento      3929 non-null   int32         
 8   sp500              3830 non-null   float64       
 9   US$Comercial       3902 non-null   float64       
 10  usaTxJuros         3827 non-null   float64       
 11  Ano                3929 non-null   int32         
 12  EBITDA             993 non-null    float64       
 13  Total Expenses     993 non-null    float64       
 14  Net Inco

In [ ]:
#####################################################################
# Convertendo o NaT para 0 e o valor de data para 1                 # 
#####################################################################
# sugestao do chatgpt para resolver o problema
#
df_combinado['dividends'] = df_combinado['dividends'].notna().astype(int)
df_combinado['jscpValor'] = df_combinado['jscpValor'].notna().astype(int)
df_combinado['jscpDataCom'] = df_combinado['jscpDataCom'].notna().astype(int)
df_combinado['jscpPagamento'] = df_combinado['jscpPagamento'].notna().astype(int)

df_combinado['sp500'] = df_combinado['sp500'].notna().astype(int)
df_combinado['US$Comercial'] = df_combinado['US$Comercial'].notna().astype(int)
df_combinado['usaTxJuros'] = df_combinado['usaTxJuros'].notna().astype(int)
df_combinado['EBITDA'] = df_combinado['EBITDA'].notna().astype(int)

df_combinado['Total Expenses'] = df_combinado['Total Expenses'].notna().astype(int)
df_combinado['Net Income'] = df_combinado['Net Income'].notna().astype(int)
df_combinado['Operating Income'] = df_combinado['Operating Income'].notna().astype(int)
df_combinado['Operating Expense'] = df_combinado['Operating Expense'].notna().astype(int)

df_combinado['Gross Profit'] = df_combinado['Gross Profit'].notna().astype(int)
df_combinado['Total Revenue'] = df_combinado['Total Revenue'].notna().astype(int)
df_combinado['Operating Revenue'] = df_combinado['Operating Revenue'].notna().astype(int)



vale_dj.head()

Operating Revenue 

### 12 - Gráficos e Analises

In [ ]:
vale_dj[vale_dj.index == '2023-08-11']

In [ ]:


# tem que ajustar, esta errado

df_august_2022 = vale_dj[(vale_dj.index >= '2021-01-01') & (vale_dj.index <= '2022-08-30')]

df_dividends = df_august_2022[df_august_2022['Dividends'] != 0]

# 
# Plotando o gráfico de linha com duas linhas: 'Close' e 'Dividends'
plt.figure(figsize=(10, 6))
plt.plot(df_august_2022.index, df_august_2022['Close'], marker='o', linestyle='-', label='Close')
plt.scatter(df_dividends.index, df_dividends['Close'], color='red', label='Dividends', zorder=5)
plt.title('Preço de Fechamento e Dividendos em Agosto de 2022')
plt.xlabel('Data')
plt.ylabel('Preço de Fechamento')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#
# Finalizada a primeira faze que tinha como objetivo montar uma 
# estrutura de dados com informacoes de dividendos e jscp
#
correlation_matrix = vale_dj.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de Correlação')
plt.show()

<p> <font color='grey'> Função para extrair informações do arquivo income</h4>
<hr>

In [ ]:
EBITDA = extract_financial_info('2022-12-31', 'EBITDA', income)
EBITDA

<p> <font color='grey'> Insere informações do balanço, que tem periodicidade anual, no arquivo da vale </p>
<hr>

<h4> <font color='purple'> Criando uma conta na nasdaq para extrair informacoes do iron ore
tambem vamos extrair a cotacao do dolar e informacoes de balanco da yfinance</h4>